# Retriever

In [ ]:
# uv add rich : 포메팅 의존성

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [11]:
# 출력 예쁘게 하기
from rich.console import Console
from rich.table import Table

console = Console()

def rich_docs(docs, max_len=140, title="Retriever Results"):
    table = Table(title=title)
    table.add_column("#", justify="right")
    table.add_column("Source")
    table.add_column("Page", justify="right")
    table.add_column("Preview")

    for i, d in enumerate(docs, 1):
        m = d.metadata or {}
        src = (m.get("source","") or "").split("/")[-1]
        page = str(m.get("page_label", m.get("page",0)+1))
        text = (d.page_content or "").strip().replace("\n", " ")
        table.add_row(str(i), src, page, (text[:max_len] + ("…" if len(text) > max_len else "")))

    console.print(table)

## retriever 설정
- 일반 RAG 기본값 : similarity or mmr
- 중복이 많을 경우 : mmr
- 그외 필터링이 필요한 경우 : search_krwargs
- 길이가 긴 경우 : compressed_retriever | parant_child
- 용어가 중요할 경우 : hybrid(vec + bm25)
- 정확도 극대화 : similarity -> rerank -> reorder

In [3]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

In [5]:
emb = OpenAIEmbeddings(model='text-embedding-3-small')

db_path = '../vectorstore/chroma_samsung'
collection_name = 'samsung_all'

vecstore = Chroma(
    persist_directory=db_path,
    collection_name=collection_name,
    embedding_function=emb
)

vecstore._collection.count()

444

In [6]:
dim_size = len(emb.embed_query('test'))
dim_size

1536

## 1. 벡터 기반 검색기(유사도/mmr/score_threshole/filter)

In [7]:
question = '삼성의 지속 가능성에 대해 알려줘'

In [ ]:
# similarity
ret_sim = vecstore.as_retriever(
    search_type = 'similarity',
    search_kwargs = {'k' : 10}
    
)

rich_docs(ret_sim.invoke(question), title='similarity')

                                                    similarity                                                     
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ ·  삼성전자주식회사 지속가능경영 웹사이트        │
│    │                                                  │      │ http://www.samsung.com/sec/sustainability/main · │
│    │                                                  │      │ 삼성전자주식회사 IR 웹사이트                     │
│    │                                                  │      │ http://www.samsung.com/sec/ir ·                  │
│    │                                                  │      │ 삼성전자주식회사 뉴…                             │
│  2 │ Sustainability_report_2024_kr.pdf                │   82 │ 비록 삼성전자주식회사는 지속가능경영보고서의     │
│    │                                                  │      │ 미래 예측 진술이 시의성 있고 합리적인 정보, 가 … │
│    │                                                  │      │ 및 믿음에 기반한다고 판단하지만, 이러한 미래     │
│    │                                                  │      │ 예측 진술(그리고 이를 이루는 정보, 가정 및       │
│    │                                                  │      │ 믿음)은  다양한 요인, 리스크, 불확실성의         │
│    │                                                  │      │ 영향권에 있으므로…                               │
│  3 │ Sustainability_report_2024_kr.pdf                │   82 │ ISAE3000을 적용했습니다. 관련 정보 ·             │
│    │                                                  │      │ 삼성전자주식회사 대표 홈페이지                   │
│    │                                                  │      │ http://www.samsung.com/sec ·  삼성전자주식회사   │
│    │                                                  │      │ 지속가능경영 웹사이트                            │
│    │                                                  │      │ http://www.samsung.com/sec/sustainabilit…        │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86              │
│    │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출    │
│    │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기     │
│    │                                                  │      │ 위해 2025년 열여덟 번째 지속가능경영보고서를     │
│    │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영   │
│    │                                                  │      │ 보고 기준인 GRI(G…                               │
│  5 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 비록 삼성전자주식회사는 지속가능경영보고서의     │
│    │                                                  │      │ 미래 예측 진술이 시의성 있고 합리적인 정보, 가 … │
│    │                                                  │      │ 및 믿음에 기반한다고 판단하지만, 이러한 미래     │
│    │                                                  │      │ 예측 진술(그리고 이를 이루는 정보, 가정 및       │
│    │                                                  │      │ 믿음)은  다양한 요인, 리스크, 불확실성의         │
│    │                                                  │      │ 영향권에 있으므로…                               │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey       │
│    │                                                  │      │ Towards   a Sustainable Future A Journey         │
│    │                                                  │      │ Towards  a Sustainable Future                    │
│  7 │ Sustainability_report_2024_kr.pdf                │   55 │ 삼성전자 지속가능경영보고서 2024 55Our Company   │
│    │                                                  │      │ AppendixMateriality Assessment Facts & Figures   │
│    │                                                  │      │ PrinciplePlanet People Facts & Figures 56      

In [18]:
# mmr
ret_mmr = vecstore.as_retriever(
    search_type = 'mmr',
    search_kwargs = {'k' : 10,
                    'fetch_k' : 20,
                    'lambda_mult' : 0.5 # 1 : 관련성, 0 : 다양성
                    }
)

rich_docs(ret_mmr.invoke(question), title='mmr')


                                                        mmr                                                        
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                                           ┃ Page ┃ Preview                                          ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ ·  삼성전자주식회사 지속가능경영 웹사이트        │
│    │                                                  │      │ http://www.samsung.com/sec/sustainability/main · │
│    │                                                  │      │ 삼성전자주식회사 IR 웹사이트                     │
│    │                                                  │      │ http://www.samsung.com/sec/ir ·                  │
│    │                                                  │      │ 삼성전자주식회사 뉴…                             │
│  2 │ Sustainability_report_2024_kr.pdf                │   82 │ 비록 삼성전자주식회사는 지속가능경영보고서의     │
│    │                                                  │      │ 미래 예측 진술이 시의성 있고 합리적인 정보, 가 … │
│    │                                                  │      │ 및 믿음에 기반한다고 판단하지만, 이러한 미래     │
│    │                                                  │      │ 예측 진술(그리고 이를 이루는 정보, 가정 및       │
│    │                                                  │      │ 믿음)은  다양한 요인, 리스크, 불확실성의         │
│    │                                                  │      │ 영향권에 있으므로…                               │
│  3 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ 삼성전자 지속가능경영보고서 2025 86              │
│    │                                                  │      │ 삼성전자주식회사는 경제·사회·환경적 가치 창출    │
│    │                                                  │      │ 성과를 다양한 이해관계자와 투명하게 소통하기     │
│    │                                                  │      │ 위해 2025년 열여덟 번째 지속가능경영보고서를     │
│    │                                                  │      │ 발간합니다. 작성 기준 본 보고서는 지속가능경영   │
│    │                                                  │      │ 보고 기준인 GRI(G…                               │
│  4 │ Samsung_Electronics_Sustainability_Report_2025_… │    1 │ 삼성전자 지속가능경영보고서 2025 A Journey       │
│    │                                                  │      │ Towards   a Sustainable Future A Journey         │
│    │                                                  │      │ Towards  a Sustainable Future                    │
│  5 │ Sustainability_report_2024_kr.pdf                │    4 │ 삼성전자 지속가능경영보고서 2024 04 Our Company  │
│    │                                                  │      │ AppendixMateriality Assessment Facts & Figures   │
│    │                                                  │      │ PrinciplePlanet People CEO 메시지 Message from   │
│    │                                                  │      │ Our CEO 주주, 고…                                │
│  6 │ Samsung_Electronics_Sustainability_Report_2025_… │   46 │ 삼성전자 지속가능경영보고서 2025 46 주요 평가    │
│    │                                                  │      │ 항목 책임있는 구매관행 거래 기본 계약서 체결     │
│    │                                                  │      │ 삼성전자는 협력회사와의 거래 기본 계약서에       │
│    │                                                  │      │ 온실가스 관리를 포함한 협력회사 행동규범 준수    │
│    │                                                  │      │ 의무를 명시함으로써  협력회사 지속가능경영 기반  │
│    │                                                  │      │ 강화를 위해 노…                                  │
│  7 │ Samsung_Electronics_Sustainability_Report_2025_… │   76 │ 삼성전자 지속가능경영보고서 2025 76 독립된       │
│    │                                                  │      │ 인증인의 인증보고서 Our Company AppendixFacts &  │
│    │                                                  │      │ Figures PrinciplePlanet People                   │
│  8 │ Sustainability_report_2024_kr.pdf                │   52 │ 삼성전자 지속가능경영보고서 2024 52Our Company   │

In [ ]:
# similarity_score_threshold
ret_threshold = vecstore.as_retriever(
    search_type = 'similarity_score_threshold',
    search_kwargs = {'k' : 8,
                    'score_threshold' : 0.4, # 유사도 기준값으로 필터링
                    }
)

rich_docs(ret_threshold.invoke(question), title='similarity_score_threshold')

                                            similarity_score_threshold                                             
┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ # ┃ Source                                           ┃ Page ┃ Preview                                           ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ Samsung_Electronics_Sustainability_Report_2025_… │   86 │ ·  삼성전자주식회사 지속가능경영 웹사이트         │
│   │                                                  │      │ http://www.samsung.com/sec/sustainability/main ·  │
│   │                                                  │      │ 삼성전자주식회사 IR 웹사이트                      │
│   │                                                  │      │ http://www.samsung.com/sec/ir ·  삼성전자주식회 … │
│   │                                                  │      │ 뉴…                                               │
└───┴──────────────────────────────────────────────────┴──────┴───────────────────────────────────────────────────┘

In [24]:
# filter

print(vecstore._collection.get(limit=1))

ret_filter = vecstore.as_retriever(
    search_kwargs = {'k' : 10,
                    'filter' : {'source': '../data/Sustainability_report_2024_kr.pdf'}
                    }
    
)

rich_docs(ret_filter.invoke(question), title='filter')

{'ids': ['sv_2024::c70cb805-6d44-4ca1-9061-27a9071dfc8e'], 'embeddings': None, 'documents': ['A Journey Towards  \na Sustainable Future\n삼성전자 지속가능경영보고서 2024'], 'uris': None, 'included': ['metadatas', 'documents'], 'data': None, 'metadatas': [{'creator': 'Adobe InDesign 15.1 (Macintosh)', 'page_label': '1', 'moddate': '2024-11-25T11:10:46+09:00', 'total_pages': 83, 'producer': 'Adobe PDF Library 15.0', 'creationdate': '2024-11-25T11:10:32+09:00', 'page': 0, 'source': '../data/Sustainability_report_2024_kr.pdf', 'trapped': '/False'}]}


                                                      filter                                                       
┏━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  # ┃ Source                            ┃ Page ┃ Preview                                                         ┃
┡━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  1 │ Sustainability_report_2024_kr.pdf │   82 │ 비록 삼성전자주식회사는 지속가능경영보고서의 미래 예측 진술이   │
│    │                                   │      │ 시의성 있고 합리적인 정보, 가정  및 믿음에 기반한다고           │
│    │                                   │      │ 판단하지만, 이러한 미래 예측 진술(그리고 이를 이루는 정보, 가 … │
│    │                                   │      │ 및 믿음)은  다양한 요인, 리스크, 불확실성의 영향권에 있으므로…  │
│  2 │ Sustainability_report_2024_kr.pdf │   82 │ ISAE3000을 적용했습니다. 관련 정보 ·  삼성전자주식회사 대표     │
│    │                                   │      │ 홈페이지   http://www.samsung.com/sec ·  삼성전자주식회사       │
│    │                                   │      │ 지속가능경영 웹사이트                                           │
│    │                                   │      │ http://www.samsung.com/sec/sustainabilit…                       │
│  3 │ Sustainability_report_2024_kr.pdf │   55 │ 삼성전자 지속가능경영보고서 2024 55Our Company                  │
│    │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet  │
│    │                                   │      │ People Facts & Figures 56     경제성과 57     사…               │
│  4 │ Sustainability_report_2024_kr.pdf │   30 │ 삼성전자 지속가능경영보고서 2024 30Our Company                  │
│    │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet  │
│    │                                   │      │ People 사회적 책임을 다하며 미래로 함께 나아갑니다. People 31 … │
│  5 │ Sustainability_report_2024_kr.pdf │    4 │ 삼성전자 지속가능경영보고서 2024 04 Our Company                 │
│    │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet  │
│    │                                   │      │ People CEO 메시지 Message from  Our CEO 주주, 고…               │
│  6 │ Sustainability_report_2024_kr.pdf │   82 │ "outlook," "believes," "estimates," "predicts," "potential,"    │
│    │                                   │      │ "continue," " preliminary,"   “추진”, “목표”, “전략”, “예정”과  │
│    │                                   │      │ 같은 용어로 미래 예측 진술을 식별할 수 있습…                    │
│  7 │ Sustainability_report_2024_kr.pdf │   82 │ 삼성전자 지속가능경영보고서 2024 82 Our Company                 │
│    │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet  │
│    │                                   │      │ People About This Report 삼성전자주식회사는 경제·사회·환…       │
│  8 │ Sustainability_report_2024_kr.pdf │   52 │ 삼성전자 지속가능경영보고서 2024 52Our Company                  │
│    │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet  │
│    │                                   │      │ People 지속가능한 미래를 위해 책임경영을 실천합니다. Principle  │
│    │                                   │      │ …                                                               │
│  9 │ Sustainability_report_2024_kr.pdf │   39 │ 삼성전자 지속가능경영보고서 2024 39 Our Company                 │
│    │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet  │
│    │                                   │      │ People 추진 체계 삼성전자는 이사회 산하 지속가능경영위원회와    │
│    │                                   │      │ 인권분야 …                                                      │
│ 10 │ Sustainability_report_2024_kr.pdf │   23 │ 삼성전자 지속가능경영보고서 2024 23 Our Company                 │
│    │                                   │      │ AppendixMateriality Assessment Facts & Figures PrinciplePlanet  │
│    │                                   │      │ People 추진방향